In [1]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi


from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

shots = 8192 
max_credits = 3 
#job_exp = execute(qc, backend=backend, shots=shots, max_credits=max_credits)
from qiskit.compiler import transpile

In [2]:
#Simulation
#Misura di tutti i qubit
n=4
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i])
    


#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
print(circuits[-1].parameters)
circuits[-1].draw(output='mpl')

#circuits[1].draw()


job = execute(circ,
            backend=BasicAer.get_backend('qasm_simulator'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range])

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 0000')
print(list(map(lambda c: 1/1024*c.get('0000', 0), counts2)))

Phi range
[0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628]
set()


Counts
[{'0000': 1024}, {'0001': 943, '0000': 81}, {'0001': 368, '0000': 656}, {'0001': 403, '0000': 621}, {'0001': 923, '0000': 101}, {'0000': 1024}, {'0001': 929, '0000': 95}, {'0001': 329, '0000': 695}, {'0001': 371, '0000': 653}, {'0001': 917, '0000': 107}]


Counts of 0000
[1.0, 0.0791015625, 0.640625, 0.6064453125, 0.0986328125, 1.0, 0.0927734375, 0.6787109375, 0.6376953125, 0.1044921875]


In [3]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 4
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')


job = execute(circ,
            backend=IBMQ.get_backend('ibmqx2'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 0000')
print(list(map(lambda c: 1/shots*c.get('0000', 0), counts2)))
job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

Phi range
[0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628]
Job Status: job has successfully run


Counts
[{'1100': 192, '0001': 3518, '0100': 454, '1101': 221, '0110': 110, '0011': 343, '1000': 302, '0101': 375, '1001': 595, '0000': 1186, '0111': 103, '1010': 91, '1011': 71, '1111': 81, '0010': 460, '1110': 90}, {'1100': 174, '0001': 1836, '0100': 735, '1101': 260, '0110': 226, '0011': 317, '1000': 282, '0101': 1048, '1001': 320, '0000': 2209, '0111': 186, '1010': 50, '1011': 57, '1111': 60, '0010': 352, '1110': 80}, {'1100': 370, '0001': 1758, '0100': 832, '1101': 215, '0110': 232, '0011': 317, '1000': 314, '0101': 993, '1001': 456, '0000': 1920, '0111': 178, '1010': 70, '1011': 64, '1111': 78, '0010': 319, '1110': 76}, {'1100': 482, '0001': 2409, '0100': 715, '1101': 252, '0110': 145, '0011': 390, '1000': 535, '0101': 254, '1001': 518, '0000': 1646, '011

In [4]:
#Run in real quantum computer - Transpile
#Misura di tutti i qubit
n = 4
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

backend = IBMQ.get_backend('ibmqx2')
trans_ghz = transpile(circ, backend)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 0000')
print(list(map(lambda c: 1/shots*c.get('0000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628]
Job Status: job has successfully run


Counts
[{'1100': 222, '0001': 3224, '0100': 455, '1101': 285, '0110': 174, '0011': 359, '1000': 320, '0101': 343, '1001': 530, '0000': 1256, '0111': 134, '1010': 106, '1011': 108, '1111': 104, '0010': 441, '1110': 131}, {'1100': 199, '0001': 1797, '0100': 709, '1101': 270, '0110': 246, '0011': 287, '1000': 313, '0101': 886, '1001': 328, '0000': 2260, '0111': 185, '1010': 72, '1011': 58, '1111': 87, '0010': 404, '1110': 91}, {'1100': 407, '0001': 1716, '0100': 754, '1101': 210, '0110': 260, '0011': 339, '1000': 317, '0101': 889, '1001': 417, '0000': 1991, '0111': 201, '1010': 84, '1011': 82, '1111': 93, '0010': 330, '1110': 102}, {'1100': 474, '0001': 2555, '0100': 590, '1101': 247, '0110': 158, '0011': 410, '1000': 460, '0101': 284, '1001': 456, '0000': 1624, 

In [2]:
#Run in real quantum computer - Transpile user
#Misura di tutti i qubit
n = 4
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 0, q[1]: 1, q[2]: 2, q[3]:4}

backend = IBMQ.get_backend('ibmqx2')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 0000')
print(list(map(lambda c: 1/shots*c.get('0000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628]
Job Status: job has successfully run


Counts
[{'0100': 491, '1000': 380, '1111': 72, '1110': 95, '0010': 323, '0011': 251, '1101': 283, '0110': 114, '0101': 393, '1011': 70, '1100': 247, '0000': 1029, '1010': 87, '0111': 89, '0001': 3539, '1001': 729}, {'0100': 803, '1000': 351, '1111': 60, '1110': 67, '0010': 285, '0011': 249, '1101': 319, '0110': 193, '0101': 1114, '1011': 41, '1100': 263, '0000': 2152, '1010': 79, '0111': 136, '0001': 1665, '1001': 415}, {'0100': 917, '1000': 379, '1111': 61, '1110': 75, '0010': 233, '0011': 304, '1101': 256, '0110': 209, '0101': 1109, '1011': 51, '1100': 468, '0000': 1632, '1010': 55, '0111': 152, '0001': 1699, '1001': 592}, {'0100': 697, '1000': 602, '1111': 113, '1110': 114, '0010': 252, '0011': 293, '1101': 295, '0110': 105, '0101': 263, '1011': 77, '1100':